In [1]:
!pip uninstall kaggle -y
!pip uninstall --upgrade.pip
!pip install kaggle==1.5.6
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle
!chmod 600 ~./kaggle/kaggle.json

Found existing installation: kaggle 1.5.12
Uninstalling kaggle-1.5.12:
  Successfully uninstalled kaggle-1.5.12

Usage:   
  pip3 uninstall [options] <package> ...
  pip3 uninstall [options] -r <requirements file> ...

no such option: --upgrade.pip
     |████████████████████████████████| 58 kB 3.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72857 sha256=bb596e03eb5ebf5ac131b5cafc3c0d14ebc9b8fb56a5af319a6faaed0aa57449
  Stored in directory: /root/.cache/pip/wheels/aa/e7/e7/eb3c3d514c33294d77ddd5a856bdd58dc9c1fabbed59a02a2b
Successfully built kaggle
chmod: cannot access '~./kaggle/kaggle.json': No such file or directory


In [2]:
!kaggle competitions download -c 2021-ai-w6-p1

 89% 82.0M/92.4M [00:00<00:00, 216MB/s]
100% 92.4M/92.4M [00:00<00:00, 211MB/s]


In [3]:
!unzip 2021-ai-w6-p1.zip

Archive:  2021-ai-w6-p1.zip
  inflating: pca_test.csv            
  inflating: pca_train.csv           
  inflating: sample_submission.csv   
  inflating: train_label.csv         


In [4]:
import pandas as pd
import numpy as np

x=pd.read_csv("pca_train.csv")
y=pd.read_csv("train_label.csv")
test=pd.read_csv("pca_test.csv")
submit=pd.read_csv("sample_submission.csv")

In [5]:
y=y["label"]

In [6]:
y

0        136
1         90
2        112
3        165
4         73
        ... 
27666      9
27667    132
27668     18
27669     41
27670    114
Name: label, Length: 27671, dtype: int64

In [7]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(x)
x_std=scaler.transform(x)
test_std=scaler.transform(test)

In [8]:
import torch

torch.manual_seed(777)

device='cuda' if torch.cuda.is_available() else 'cpu'
if device=='cuda':
  torch.cuda.manual_seed_all(777)

In [9]:
x=np.array(x)
y=np.array(y)
test=np.array(test)
x=torch.FloatTensor(x)
y=torch.LongTensor(y)
test=torch.FloatTensor(test)

In [10]:
y.unique()

tensor([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,  27,
         28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,  40,  41,
         42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,  53,  54,  55,
         56,  57,  58,  59,  60,  61,  62,  63,  64,  65,  66,  67,  68,  69,
         70,  71,  72,  73,  74,  75,  76,  77,  78,  79,  80,  81,  82,  83,
         84,  85,  86,  87,  88,  89,  90,  91,  92,  93,  94,  95,  96,  97,
         98,  99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111,
        112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125,
        126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139,
        140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167,
        168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 1

In [11]:
linear1=torch.nn.Linear(256,512)
linear2=torch.nn.Linear(512,1024)
linear3=torch.nn.Linear(1024,512)
linear4=torch.nn.Linear(512,256)
linear5=torch.nn.Linear(256,198)
relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(0.1)

In [12]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)

Parameter containing:
tensor([[ 0.0190,  0.0700,  0.0400,  ...,  0.0708, -0.0719, -0.0789],
        [ 0.0725,  0.0705, -0.0799,  ...,  0.0165, -0.0181,  0.0221],
        [ 0.0632, -0.0180,  0.1087,  ..., -0.0072, -0.0147, -0.0164],
        ...,
        [ 0.1076, -0.1015, -0.0166,  ..., -0.0538, -0.0706,  0.0436],
        [-0.0498, -0.0055, -0.1119,  ...,  0.0653,  0.1098, -0.0062],
        [-0.0476,  0.0976, -0.0903,  ...,  0.0125, -0.1027, -0.1106]],
       requires_grad=True)

In [13]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4,relu,dropout,linear5).to(device)
optimizer = torch.optim.Adam(params=model.parameters(),lr=0.001)
loss=torch.nn.CrossEntropyLoss().to(device)

In [14]:
for stop in range(90):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.to(device))
  cost.backward()
  optimizer.step()
  if stop%10==0:
    print(stop,cost.item())

0 5.738467216491699
10 3.1765849590301514
20 2.1777050495147705
30 1.8026551008224487
40 1.5578157901763916
50 1.3631309270858765
60 1.2231882810592651
70 1.0891855955123901
80 0.9661682844161987


In [15]:
prediction=list()
with torch.no_grad():
  model.eval()
  pred=model(test.to(device))
  a,b=torch.max(pred,dim=1)
  prediction.extend(b.tolist())

In [16]:
submit["label"]=prediction
submit.to_csv("submit.csv",index=False)

In [18]:
!kaggle competitions submit -c 2021-ai-w6-p1 -f submit.csv -m "Message"

100% 116k/116k [00:02<00:00, 54.4kB/s]
Successfully submitted to DNN을 이용한 원자력발전소 상태 판단